In [3]:
!pip -q install pandas numpy matplotlib

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 200)

In [5]:
df = pd.DataFrame({
    "Name": [" Sanju", "Kamal", None, "  Nimal  ", "sanju"],
    "Age": ["25", " 30", "?", None, "27 "],
    "Salary": ["£2,500", "3000", None, "4,200", " 3500 "],
    "City": ["London", "london ", "Coventry", None, "LONDON"],
    "Joined": ["2024-01-10", "10/02/2024", "2024.03.05", "", None]
})
df

,Name,Age,Salary,City,Joined
0,Sanju,25,"£2,500",London,2024-01-10
1,Kamal,30,3000,london,10/02/2024
2,None,?,None,Coventry,2024.03.05
3,Nimal,None,"4,200",None,
4,sanju,27,3500,LONDON,None


In [7]:
def audit(df):
    print("Shape:", df.shape)
    display(df.head())
    print("\nMissing values per column:")
    print(df.isna().sum())
    print("\nData types:")
    print(df.dtypes)
    print("\nDuplicates:", df.duplicated().sum())

audit(df)

Shape: (5, 5)


,Name,Age,Salary,City,Joined
0,Sanju,25,"£2,500",London,2024-01-10
1,Kamal,30,3000,london,10/02/2024
2,None,?,None,Coventry,2024.03.05
3,Nimal,None,"4,200",None,
4,sanju,27,3500,LONDON,None



Missing values per column:
Name      1
Age       1
Salary    1
City      1
Joined    1
dtype: int64

Data types:
Name      object
Age       object
Salary    object
City      object
Joined    object
dtype: object

Duplicates: 0


In [8]:
import re

def clean_text(s):
    if pd.isna(s):
        return np.nan
    s = str(s).strip().lower()
    s = re.sub(r"\s+", " ", s)   # collapse multiple spaces
    return s

def clean_number(x):
    if pd.isna(x):
        return np.nan
    x = str(x).strip()
    x = x.replace("£", "").replace(",", "")
    if x in ["", "?", "nan", "None"]:
        return np.nan
    try:
        return float(x)
    except:
        return np.nan

def clean_date(x):
    if pd.isna(x) or str(x).strip() == "":
        return pd.NaT
    return pd.to_datetime(x, errors="coerce", dayfirst=True)

In [9]:
df_clean = df.copy()

# text columns
for col in ["Name", "City"]:
    df_clean[col] = df_clean[col].apply(clean_text)

# numeric columns
df_clean["Age"] = df_clean["Age"].apply(clean_number)
df_clean["Salary"] = df_clean["Salary"].apply(clean_number)

# date column
df_clean["Joined"] = df_clean["Joined"].apply(clean_date)

df_clean

,Name,Age,Salary,City,Joined
0,sanju,25.0,2500.0,london,2024-10-01
1,kamal,30.0,3000.0,london,2024-02-10
2,NaN,NaN,NaN,coventry,2024-05-03
3,nimal,NaN,4200.0,NaN,NaT
4,sanju,27.0,3500.0,london,NaT


In [10]:
df_filled = df_clean.copy()

for col in df_filled.columns:
    if pd.api.types.is_numeric_dtype(df_filled[col]):
        df_filled[col] = df_filled[col].fillna(df_filled[col].median())
    elif pd.api.types.is_datetime64_any_dtype(df_filled[col]):
        pass
    else:
        df_filled[col] = df_filled[col].fillna("unknown")

df_filled

,Name,Age,Salary,City,Joined
0,sanju,25.0,2500.0,london,2024-10-01
1,kamal,30.0,3000.0,london,2024-02-10
2,unknown,27.0,3250.0,coventry,2024-05-03
3,nimal,27.0,4200.0,unknown,NaT
4,sanju,27.0,3500.0,london,NaT


In [11]:
display(df_filled.describe(include="all"))

,Name,Age,Salary,City,Joined
count,5,5.000000,5.000000,5,3
unique,4,NaN,NaN,3,NaN
top,sanju,NaN,NaN,london,NaN
freq,2,NaN,NaN,3,NaN
mean,NaN,27.200000,3290.000000,NaN,2024-05-25 16:00:00
min,NaN,25.000000,2500.000000,NaN,2024-02-10 00:00:00
25%,NaN,27.000000,3000.000000,NaN,2024-03-22 12:00:00
50%,NaN,27.000000,3250.000000,NaN,2024-05-03 00:00:00
75%,NaN,27.000000,3500.000000,NaN,2024-07-17 12:00:00
max,NaN,30.000000,4200.000000,NaN,2024-10-01 00:00:00


In [12]:
df_filled.to_csv("cleaned_data.csv", index=False)
print("Saved:", "cleaned_data.csv")

Saved: cleaned_data.csv
